In [1]:
import math
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

words = open("60knames.csv",'r').read().splitlines()[1:] # col name removed
words = [word for word in words if not any(char in word for char in ['/','"',',','-',".","/",";",'±','Ã'])]

# words = open("names.txt",'r').read().splitlines()
inp = []
out = []

all_char = sorted(set(".".join(words)))
stoi = {ch:i for i,ch in enumerate(all_char)}
itos = {v:k for k,v in stoi.items()}

clen = 2
for w in words:
    context = '.' * clen
    w =  w + '.'
    for cur_ch in w:
        cur_inp = [stoi[ch] for ch in context]
        cur_inp[-1] += 27
        inp.append(cur_inp)
        out.append(stoi[cur_ch])
        context = context[1:] + cur_ch
    
inp_tri = math.ceil(0.8*len(inp))
inp_devi = math.ceil(0.9*len(inp))
inp_tr = torch.tensor(inp[0:inp_tri])
out_tr = torch.tensor(out[0:inp_tri])
inp_dev =torch.tensor(inp[inp_tri:inp_devi])
out_dev =torch.tensor(out[inp_tri:inp_devi])
inp_tst =torch.tensor(inp[inp_devi:])
out_tst =torch.tensor(out[inp_devi:])

inp_tr  = F.one_hot(inp_tr,num_classes = 54).float()
inp_tr = inp_tr.sum(dim=1)

inp_tr[:10] ,out_tr[:10]

(tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0.

In [2]:
g = torch.Generator().manual_seed(16)
W = torch.randn((54,27),requires_grad = True,generator = g)

In [ ]:
for i in range(1000):
    logits = inp_tr @ W
    loss = F.cross_entropy(logits,out_tr)
    W.grad = None
    loss.backward()
    print(f'loss = {loss.item()} iter = {i}')
    W.data += -.1 * W.grad

loss = 2.2171647548675537 iter = 0
loss = 2.217155933380127 iter = 1
loss = 2.217146873474121 iter = 2
loss = 2.2171378135681152 iter = 3
loss = 2.2171289920806885 iter = 4
loss = 2.2171196937561035 iter = 5
loss = 2.217111110687256 iter = 6
loss = 2.21710205078125 iter = 7
loss = 2.217092752456665 iter = 8
loss = 2.217083692550659 iter = 9
loss = 2.2170748710632324 iter = 10
loss = 2.2170660495758057 iter = 11
loss = 2.2170569896698 iter = 12
loss = 2.217048168182373 iter = 13
loss = 2.217039108276367 iter = 14
loss = 2.2170300483703613 iter = 15
loss = 2.2170212268829346 iter = 16
loss = 2.2170121669769287 iter = 17
loss = 2.217003345489502 iter = 18
loss = 2.216994047164917 iter = 19
loss = 2.2169852256774902 iter = 20
loss = 2.2169764041900635 iter = 21
loss = 2.2169673442840576 iter = 22
loss = 2.216958522796631 iter = 23
loss = 2.216949462890625 iter = 24
loss = 2.216940402984619 iter = 25
loss = 2.2169315814971924 iter = 26
loss = 2.2169225215911865 iter = 27
loss = 2.2169134616

In [ ]:
for _ in range(20):
  x1,x2 = 0,27
  out = '.'
  while True:
    inp = F.one_hot(torch.tensor([x1]),num_classes=54).float() + F.one_hot(torch.tensor([x2]),num_classes = 54).float()
    logits = inp @ W
    prob = F.softmax(logits,dim=1) 
    x1,x2 = x2,torch.multinomial(prob,num_samples = 1,generator = g,replacement=True)
    out += itos[x2.item()]
    if itos[x2.item()]=='.':
      break
  print(out)
# f.one_hot(torch.tensor([x1]),num_classes=54) + f.one_hot(torch.tensor([x2],num_classes = 54))